<a href="https://colab.research.google.com/github/owos/Talks/blob/main/IndabaX-NG24/IndabaX_ASR_Master_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Author: Abraham Owodunni.
#### Adapted from [here](https://huggingface.co/blog/fine-tune-whisper)

# Fine-Tune Whisper For AfriSpeech Competition


Notes: The data is ~120gb and the free version  google colab can only handle 78.2GB, so we'll need to use data streaming mode to load the data.

In this Colab, we present a step-by-step guide on how to fine-tune Whisper
for the AfriSpeech competition using Hugging Face 🤗 Transformers.

## Introduction

Whisper is a pre-trained model for automatic speech recognition (ASR)
published in [September 2022](https://openai.com/blog/whisper/) by the authors
Alec Radford et al. from OpenAI. Unlike many of its predecessors, such as
[Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), which are pre-trained
on un-labelled audio data, Whisper is pre-trained on a vast quantity of
**labelled** audio-transcription data, 680,000 hours to be precise.
This is an order of magnitude more data than the un-labelled audio data used
to train Wav2Vec 2.0 (60,000 hours). What is more, 117,000 hours of this
pre-training data is multilingual ASR data. This results in checkpoints
that can be applied to over 96 languages, many of which are considered
_low-resource_.

When scaled to 680,000 hours of labelled pre-training data, Whisper models
demonstrate a strong ability to generalise to many datasets and domains.
The pre-trained checkpoints achieve competitive results to state-of-the-art
ASR systems, with near 3% word error rate (WER) on the test-clean subset of
LibriSpeech ASR and a new state-of-the-art on TED-LIUM with 4.7% WER (_c.f._
Table 8 of the [Whisper paper](https://cdn.openai.com/papers/whisper.pdf)).
The extensive multilingual ASR knowledge acquired by Whisper during pre-training
can be leveraged for other low-resource languages; through fine-tuning, the
pre-trained checkpoints can be adapted for specific datasets and languages
to further improve upon these results. We'll show just how Whisper can be fine-tuned
for low-resource languages in this Colab.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 1:</b> Whisper model. The architecture
follows the standard Transformer-based encoder-decoder model. A
log-Mel spectrogram is input to the encoder. The last encoder
hidden states are input to the decoder via cross-attention mechanisms. The
decoder autoregressively predicts text tokens, jointly conditional on the
encoder hidden states and previously predicted tokens. Figure source:
<a href="https://openai.com/blog/whisper/">OpenAI Whisper Blog</a>.</figcaption>
</figure>

The Whisper checkpoints come in five configurations of varying model sizes.
The smallest four are trained on either English-only or multilingual data.
The largest checkpoint is multilingual only. All nine of the pre-trained checkpoints
are available on the [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). The
checkpoints are summarised in the following table with links to the models on the Hub:

| Size   | Layers | Width | Heads | Parameters | English-only                                         | Multilingual                                      |
|--------|--------|-------|-------|------------|------------------------------------------------------|---------------------------------------------------|
| tiny   | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)  |
| base   | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)   |
| small  | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)  |
| medium | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium) |
| large  | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)  |

For demonstration purposes, we'll fine-tune the multilingual version of the
[`"small"`](https://huggingface.co/openai/whisper-small) checkpoint with 244M params (~= 1GB).
As for our data, we'll train and evaluate our system on a low-resource language
taken from the [AfriSpeech](https://huggingface.co/datasets/tobiolatunji/afrispeech-200)
dataset. We'll show that with as little as 8 hours of fine-tuning data, we can achieve
strong performance in this language.

------------------------------------------------------------------------

\\({}^1\\) The name Whisper follows from the acronym “WSPSR”, which stands for “Web-scale Supervised Pre-training for Speech Recognition”.

## Prepare Environment

First of all, let's try to secure a decent GPU for our Colab! Unfortunately, it's becoming much harder to get access to a good GPU with the free version of Google Colab. However, with Google Colab Pro one should have no issues in being allocated a V100 or P100 GPU.

To get a GPU, click _Runtime_ -> _Change runtime type_, then change _Hardware accelerator_ from _None_ to _GPU_.

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 17 11:48:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Next, we need to update the Unix package `ffmpeg` to version 4:

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Ign:14 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ub

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of our model. Finally, we'll
use `gradio` to build a flashy demo of our fine-tuned model.

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
#!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4hp0rppg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4hp0rppg
  Resolved https://github.com/huggingface/transformers to commit a8eb4f79f946c5785f0e91b356ce328248916a05
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6553273 sha256=44f76d4369a7abb027d33832a0fdfd917c9ce30f728c0d856ce229b7ec39fd6b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vit6o4ed/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simp

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!

Linking the notebook to the Hub is straightforward - it simply requires entering your
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple.
We can download and prepare the Afrispeech splits in just one line of code.



In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "tobiolatunji/afrispeech-200", "all", streaming=True
)

Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). AfriSpeech contains additional
metadata information, such as `accent` and `domain`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

afrispeech = `load_dataset("tobiolatunji/afrispeech-200", 'yoruba', split="validation", streaming=True)`  from the list of accents or `all`  for the entire dataset

Great! Now we can take a listen of what the audio sounds like and print the text:

In [ ]:
## Use the Naija Voices Dataset here:
dataset = load_dataset("naijavoices/naijavoices-dataset","hausa-batch-1", streaming=True)


In [ ]:
k =next(iter(dataset["train"]))

Reading metadata...: 58000it [00:02, 25699.89it/s]


In [ ]:
k

{'speaker_id': 'b545a4ca235a7b72688a1c0b3eb6bde6',
 'path': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad.wav',
 'audio_id': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad',
 'audio': {'path': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad.wav',
  'array': array([0.0007019 , 0.00106812, 0.00137329, ..., 0.00540161, 0.00411987,
         0.00057983]),
  'sampling_rate': 44100},
 'transcript': '1) INFECTIOUS DISEASE / CELLULITIS :',
 'age_group': '26-40',
 'gender': 'Male',
 'accent': 'yoruba',
 'domain': 'clinical',
 'country': 'US',
 'duration': 3.2239909297052165}

In [ ]:
import IPython.display as ipd

#sample = next(iter(dataset))
audio = k["audio"]

print(k["transcript"])
ipd.Audio(data=audio["array"], autoplay=False, rate=audio["sampling_rate"])

1) INFECTIOUS DISEASE / CELLULITIS :


In [ ]:
dataset_head = dataset['train'].take(2)

list(dataset_head)

Reading metadata...: 58000it [00:01, 30782.27it/s]


[{'speaker_id': 'b545a4ca235a7b72688a1c0b3eb6bde6',
  'path': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad.wav',
  'audio_id': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad',
  'audio': {'path': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad.wav',
   'array': array([0.0007019 , 0.00106812, 0.00137329, ..., 0.00540161, 0.00411987,
          0.00057983]),
   'sampling_rate': 44100},
  'transcript': '1) INFECTIOUS DISEASE / CELLULITIS :',
  'age_group': '26-40',
  'gender': 'Male',
  'accent': 'yoruba',
  'domain': 'clinical',
  'country': 'US',
  'duration': 3.2239909297052165},
 {'speaker_id': 'b545a4ca235a7b72688a1c0b3eb6bde6',
  'path': '69f994a6-951a-497a-8e55-6003e1625b54/5790a01bf62c2ce051c9f2fdc42ea862.wav',
  'audio_id': '69f994a6-951a-497a-8e55-6003e1625b54/5790a01bf62c2ce051c9f2fdc42ea862',
  'audio': {'path': '69f994a6-951a-497a-8e55-6003e1625b54/5790a01bf62c2ce051c9f2fdc42ea862.wav',
   'array': array(

In [ ]:
#sample = next(iter(dataset))
sample =list(dataset_head)[1]
audio = sample["audio"]

print(sample["transcript"])
ipd.Audio(data=audio["array"], autoplay=False, rate=audio["sampling_rate"])

Reading metadata...: 58000it [00:01, 31000.16it/s]


In this 2 week interval he denied having CP / chest discomfort or difficulty breathing, no cough or hemoptysis .


### Optional: you can run the cell below and drop some of the unused columns

In [ ]:

dataset_cln = dataset.remove_columns(['speaker_id', 'path', 'age_group', 'gender', 'accent', 'domain', 'country', 'duration'])


print(dataset_cln)

{'train': <datasets.iterable_dataset.IterableDataset object at 0x7ff2acb640a0>, 'validation': <datasets.iterable_dataset.IterableDataset object at 0x7ff2accf4220>}


## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages: </br>
1) A feature extractor which pre-processes the raw audio-inputs </br>
2) The model which performs the sequence-to-sequence mapping </br>
3) A tokenizer which post-processes the model outputs to text format </br>

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For English, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to
specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences:

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

### Prepare Data

Let's print the first example of the AfriSpeech dataset to see
what form the data is in:

In [ ]:
print(next(iter(dataset_cln["train"])))

Reading metadata...: 58000it [00:01, 29326.58it/s]


{'audio_id': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad', 'audio': {'path': None, 'array': array([0.0007019 , 0.00106812, 0.00137329, ..., 0.00540161, 0.00411987,
       0.00057983]), 'sampling_rate': 44100}, 'transcript': '1) INFECTIOUS DISEASE / CELLULITIS :'}


Since
our input audio is sampled at 44100Hz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
from datasets import Audio

dataset_cln = dataset_cln.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(next(iter(dataset_cln["train"])))

Reading metadata...: 58000it [00:01, 30478.72it/s]


{'audio_id': '3e6b8512-8be3-4f56-978d-b9d276b50cb3/32278ae750735e11c1d5c4ff346b9dad', 'audio': {'path': None, 'array': array([ 0.00076005,  0.00114808,  0.00449508, ..., -0.00537868,
        0.00115363,  0.        ]), 'sampling_rate': 16000}, 'transcript': '1) INFECTIOUS DISEASE / CELLULITIS :'}


Now we can write a function to prepare our data for the model:
1. We load the resampled audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method.

In [ ]:
dataset_cln = dataset_cln.map(prepare_dataset)


## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it
to transcribe speech in English.

### Competition related issue: Avoiding errors during evaluation

Since you will need to predict on the validation audios to create a submission file, the Validation set transcripts (text) have been removed from HF to enable scoring on Zindi. Because there are no transcripts, you cannot use the HF validation set to evaluate your models till the competiton is over. Passing the validation set to `eval_dataset` (see below) using the huggingface trainer will lead to errors.

```
from transformers import Seq2SeqTrainer
  trainer = Seq2SeqTrainer(
  args=training_args,
  model=model,
  train_dataset=dataset_cln['train'],
  eval_dataset=dataset_cln['validation'], # this will lead to errors since validation transcripts have been removed
  data_collator=data_collator,
  compute_metrics=compute_metrics,
  tokenizer=processor.feature_extractor,
)
```

Since the ground truth transcripts in `dataset_cln['validation']` are empty, when your model predicts a transcript, it will compare to an empty ground truth and throw an error. We need a workaround till the end of the competition.

To evaluate your model during training, you should resplit your training data into train and dev as shown below:


In [ ]:
val_dataset = dataset_cln['train'].take(3000)
train_dataset = dataset_cln['train'].skip(3000)

dataset_cln['train'] = train_dataset
dataset_cln['validation'] = val_dataset

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this
is trivial through use of 🤗 Transformers!

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./dsn_afrispeech",  # change to a repo name of your choice dsn_afrispeech
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,  #try 4 and see if it crashes
    predict_with_generate=True
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    #push_to_hub=True,
)

PyTorch: setting up devices


**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_cln['train'],
    eval_dataset=dataset_cln['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [ ]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./dsn_afrispeech/preprocessor_config.json
tokenizer config file saved in ./dsn_afrispeech/tokenizer_config.json
Special tokens file saved in ./dsn_afrispeech/special_tokens_map.json
added tokens file saved in ./dsn_afrispeech/added_tokens.json


### Training

Training will take a number of hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB.
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [ ]:

##this empties the GPU
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3
  Number of trainable parameters = 241734912


config: all


Reading metadata...: 58000it [00:01, 38027.73it/s]
The following columns in the training set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: audio, transcript. If audio, transcript are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.


Step,Training Loss,Validation Loss,Wer
2,4.360000,3.643086,107.534247


***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 8


config: all


Reading metadata...: 3231it [00:00, 30804.72it/s]
The following columns in the evaluation set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: audio, transcript. If audio, transcript are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
Saving model checkpoint to ./dsn_afrispeech/checkpoint-2
Configuration saved in ./dsn_afrispeech/checkpoint-2/config.json
Configuration saved in ./dsn_afrispeech/checkpoint-2/generation_config.json
Model weights saved in ./dsn_afrispeech/checkpoint-2/pytorch_model.bin
Feature extractor saved in ./dsn_afrispeech/checkpoint-2/preprocessor_config.json


config: all


Reading metadata...: 58000it [00:01, 53331.04it/s]


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./dsn_afrispeech/checkpoint-2 (score: 107.53424657534248).


TrainOutput(global_step=3, training_loss=3.6104933420817056, metrics={'train_runtime': 30.151, 'train_samples_per_second': 0.796, 'train_steps_per_second': 0.099, 'total_flos': 5194537205760000.0, 'train_loss': 3.6104933420817056, 'epoch': 1.33})

###saving the Model to be loader to be used later in the future.

In [ ]:
trainer.save_model(training_args.output_dir)

Saving model checkpoint to ./dsn_afrispeech
Configuration saved in ./dsn_afrispeech/config.json
Configuration saved in ./dsn_afrispeech/generation_config.json
Model weights saved in ./dsn_afrispeech/pytorch_model.bin
Feature extractor saved in ./dsn_afrispeech/preprocessor_config.json


In [ ]:
processor.save_pretrained("./path_to_saved_model")

Feature extractor saved in ./path_to_saved_model/preprocessor_config.json
tokenizer config file saved in ./path_to_saved_model/tokenizer_config.json
Special tokens file saved in ./path_to_saved_model/special_tokens_map.json
added tokens file saved in ./path_to_saved_model/added_tokens.json


#Pushing your model to HF hub
The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

When we are done, We can submit our checkpoint to the [`hf-speech-bench`](https://huggingface.co/spaces/huggingface/hf-speech-bench) on push by setting the appropriate key-word arguments (kwargs):

In [ ]:
kwargs = {
    "dataset_tags": "tobiolatunji/afrispeech-200",
    "dataset": "AfriSpeech",  # a 'pretty' name for the training dataset
    "dataset_args": "config: en, split: test",
    "language": "en",
    "model_name": "Whisper Small En - Owos",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
#trainer.push_to_hub(**kwargs)

#Predictions



In [ ]:
sample = next(iter(dataset_cln['validation']))['audio']
device = "cuda" if torch.cuda.is_available() else "cpu"
input_features = processor(next(iter(dataset_cln['validation']))['audio']["array"], sampling_rate=next(iter(dataset_cln['validation']))['audio']["sampling_rate"], return_tensors="pt").input_features.to(device)
input_features.to(device)
model.to(device)
# generate token ids

predicted_ids = model.generate(input_features)

# decode token ids to text

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)


transcription


config: all


Reading metadata...: 3231it [00:00, 7233.98it/s]


config: all


Reading metadata...: 3231it [00:00, 10680.48it/s]


config: all


Reading metadata...: 3231it [00:00, 9548.75it/s]
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1284: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> We should be asking ourselves whether we want a heavy-handed approach or a ham-production approach, she added. Full stop.<|endoftext|>']

In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

[' We should be asking ourselves whether we want a heavy-handed approach or a ham-production approach, she added. Full stop.']

#### Loading the model from your local PC

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained(training_args.output_dir)
model = WhisperForConditionalGeneration.from_pretrained(training_args.output_dir)
model.to(device)

loading configuration file ./dsn_afrispeech/preprocessor_config.json
Feature extractor WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

loading file vocab.json
loading file tokenizer.json
loading file merges.txt
loading file normalizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding <|startoftranscript|> to the vocabulary
Adding <|en|> to the vocabulary
Adding <|zh|> to the vocabulary
Adding <|de|> to the vocabulary
Adding <|es|> to the vocabulary
Adding <|ru|> to the vocabulary
Adding <|ko|> to the vocabulary
Adding <|fr|> to the vocabulary
Adding <|ja|> to the vocabulary
Adding <|pt|> to the vocabulary
Adding

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0): WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
# generate token ids

predicted_ids = model.generate(input_features)
#decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1284: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> We should be asking ourselves whether we want a heavy-handed approach or a ham-production approach, she added. Full stop.<|endoftext|>']

In [ ]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

[' We should be asking ourselves whether we want a heavy-handed approach or a ham-production approach, she added. Full stop.']

#Making Final Submission with 10 samples
* Load the data,
* Load the processor,
* Load the model and predict


In [ ]:
test = dataset_cln['validation'].take(10)

In [ ]:
def make_preds(batch):

    sample = batch['audio']
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features.to(device)
    input_features.to(device)
    # generate token ids

    predicted_ids = model.generate(input_features)


    # decode token ids to text

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    batch['preds'] = transcription[0]
    return batch


In [ ]:
result = test.map(make_preds)

In [ ]:
from operator import itemgetter
results = [itemgetter('audio_id', 'preds')(i) for i in list(iter(result))]


Reading metadata...: 3231it [00:00, 9294.70it/s]
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1284: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
import pandas as pd
df = pd.DataFrame(results, columns=['ID', 'Test'])
df

,ID,Test
0,92d2b94e-3e31-40be-b479-50062e90d1e4/04c03ed2a...,We should be asking ourselves whether we want...
1,127bdba1-8bc8-44a4-9c37-8e5eef438450/86895d1ce...,Other sagittal planes parallel to this off-ce...
2,de0b1ee6-46f1-4eb6-a747-476d5b7aace0/740ddb284...,kubakiloma is an intercreniol mass occurring ...
3,c982760f-b30b-4888-9606-e5dd555b3d15/0dbb2b3d0...,Continue to use LGE amount seras from OpenABD.
4,3d7543cc-b3b4-4347-9018-70d770ac2303/bf0583a19...,Some of these roots give rise to smaller bran...
5,3842e8b6-c7bf-422c-85d2-279de4b996eb/3c6b9ae10...,just a patient's environment to promote comfort.
6,97bbfbfb-e206-46df-a69b-0aecc0165ac2/0dcf5cb5c...,kwa kwa kwa kwa kwa kwa kwa kwa kwa kwa kwa k...
7,4d49048d-fa78-4a81-aa81-db6c2008ea09/a28b0f56b...,When stress activates the sympathetic nervous...
8,e80c5101-4c8e-490b-bf7d-04bc453063c0/8cfc67a8f...,Rickets in growing children too.
9,8921c6ea-b94e-4094-bf39-f1889a6b3361/8d4a250f9...,differences between regeneration and fibroces...


## Closing Remarks

In this Class, we covered a step-by-step guide on fine-tuning Whisper for multilingual ASR
using 🤗 Datasets, Transformers and the Hugging Face Hub. For more details on the Whisper model, the AfriSpeech dataset and the theory behind fine-tuning, refere to the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper). If you're interested in fine-tuning other
Transformers models, both for English and multilingual ASR, be sure to check out the
examples scripts at [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).